---
title: "Working with Geospatial Raster Data"
subtitle: "From raw rasters to model‑ready embeddings, step by step"
editor_options: 
  chunk_output_type: console
jupyter: geoai
format: 
    html:
        toc: true
        toc-depth: 2
        code-fold: show
---


# Introduction

In this session, we learn how to transform raw geospatial raster data into model‑ready embeddings. We’ll work with a tiny multi‑band GeoTIFF, build small, deterministic processing steps, and show visible outputs after every step. The goal is to build intuition that mirrors LLM pipelines (tokenization → embeddings) while grounding everything in images (patches → embeddings).

## Learning Objectives
- Load and inspect small multi‑band rasters (CRS, resolution, dtype, bands)
- Normalize and organize bands for consistent modeling
- Extract fixed‑size patches
- Add contextual features (location/time “tokens”)
- Sample training patches correctly and create patch embeddings with shapes you can verify

## Session Roadmap

This session follows a complete data processing pipeline organized into three logical phases that transform raw satellite imagery into model-ready embeddings:


```{mermaid}
flowchart TD
    subgraph Phase1 ["🔬 Phase 1: Pixel-Level Pre-processing"]
        direction LR
        A["🛰️ Raw Data<br/>Inspection<br/>Section 1.1"] --> B["📐 Band<br/>Normalization<br/>Section 1.2"]
        B --> C["🔬 Dimensionality<br/>Reduction<br/>Section 1.3"]
    end
    
    subgraph Phase2 ["🗺️ Phase 2: Spatial-Structure Processing"]
        direction LR
        D["✂️ Patch<br/>Extraction<br/>Section 2.1"] --> E["🏷️ Patch<br/>Metadata<br/>Section 2.2"]
        E --> F["🌍 Spatial-Temporal<br/>Context<br/>Section 2.3"]
        F --> G["📊 Training<br/>Sampling<br/>Section 2.4"]
    end
    
    subgraph Phase3 ["🧠 Phase 3: Model-Ready Processing"]
        direction LR
        H["🧠 Patch<br/>Embeddings<br/>Section 3.1"] --> I["📍 Positional<br/>Encoding<br/>Section 3.2"]
    end
    
    Phase1 --> Phase2
    Phase2 --> Phase3
    
    style Phase1 fill:#e3f2fd
    style Phase2 fill:#fff3e0  
    style Phase3 fill:#e8f5e8
```


## Setting Up

Before we get started, let's setup our computational environment by loading necessary libraries and seeding our computations.


### Load required libraries
Why these imports: file paths (Path), randomness (random, numpy), and plotting (matplotlib) are used throughout the session.


In [ ]:
#| echo: true
import os, random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
print("Imports OK")

### Set Random Number Generation (RNG) seeds

We use fixed seeds to ensure that our randomness is consistent every time we run our example code. This allows your results to match the rendered output on the course site.


In [ ]:
#| echo: true
RNG_SEED = 42
os.environ["PYTHONHASHSEED"] = str(RNG_SEED)
random.seed(RNG_SEED)
np.random.seed(RNG_SEED)
print("Seeds set:", RNG_SEED)

### Define data paths and constants
We’ll store the sample raster in a local `data/` directory and set a default patch size.


In [ ]:
#| echo: true
DATA_DIR = Path("../../data"); DATA_DIR.mkdir(exist_ok=True)
DATA_PATH = DATA_DIR / "landcover_sample.tif"
PATCH_SIZE = 64
print("DATA_PATH:", DATA_PATH)
print("PATCH_SIZE:", PATCH_SIZE)

If the file is missing, we’ll download a small sample.


In [ ]:
#| echo: true
import urllib.request
if not DATA_PATH.exists():
    url = "https://raw.githubusercontent.com/kellycaylor/geoAI/main/data/landcover_sample.tif"
    print("Downloading sample raster…")
    urllib.request.urlretrieve(url, DATA_PATH)
print("Exists:", DATA_PATH.exists(), "\nSize (KB):", round(DATA_PATH.stat().st_size/1024,1))

---

# 🔬 Phase 1: Pixel-Level Pre-processing

*In this phase, we work with the full image and operate on individual pixels. We'll develop code to read our raw data, normalize it for consistent model training, and perform dimensionality reduction before moving to spatial operations.*

## 1.1 🛰️ Raw Data Inspection

### Reading Geospatial Data

Before we start working with the data in Python, it’s important to understand what kinds of information a geospatial raster dataset contains.

A single raster file stores **two broad categories of information**:

1. **Pixel values** – the actual measurements.  
   - Each pixel has one or more **bands** (channels) that store numeric values.  
   - Bands might represent reflectance in different wavelengths, elevation, temperature, or derived indices like NDVI.  
   - Pixel values are stored in a numeric data type (e.g., integers or floats) and have a defined range (min/max).

2. **Spatial metadata** – information that describes where and how those measurements fit on Earth.  
   - **Location reference** — how pixel positions map to real-world coordinates.  
   - **Pixel resolution** — the width and height of each pixel in ground units (e.g., meters).  
   - **Grid layout and orientation** — how rows and columns are aligned in space.  
   - **Coordinate system** — the mathematical model that defines positions on Earth (projection).  
   - **Transform** — a mapping from pixel coordinates (row/col) to real-world coordinates (x/y).

When we open a raster with `rasterio`, we can access **both** the measurement values and the spatial metadata.  This lets us work with the data not just as an image, but as a set of geographically-anchored measurements we can relate to location, time, and other datasets. Let's take a look at our sample image. 

### Rasterio image reading and metadata inspection


In [ ]:
# | echo: true
import rasterio as rio
import pyproj

with rio.open(DATA_PATH) as src:
    arr = src.read()                          # (bands, height, width)
    band_count, height, width = arr.shape
    dtype = arr.dtype

    # CRS and EPSG
    crs = src.crs
    epsg = crs.to_epsg()
    transform = src.transform  # Save transform for later use

    # Get a human-readable CRS name
    crs_info = pyproj.CRS.from_user_input(crs)
    crs_name = crs_info.name

    # Resolution in CRS units
    res_x, res_y = src.res

# --- Print summary ---
print(f"Shape: {arr.shape}")
print(f"Bands: {band_count}  |  Height: {height} px  Width: {width} px |  Data type: {dtype}")
print(f"CRS: {crs_name}" + (f" (EPSG:{epsg})" if epsg else ""))
print(f"Resolution: {res_x:.2f} × {res_y:.2f} ground units per pixel")
print("Per-band min/max:")
for i, b in enumerate(arr, start=1):
    print(f"  Band {i}: {float(b.min()):.3f} – {float(b.max()):.3f}")

Notice how the output gives us the fundamental "identity" of our geospatial data. The `arr.shape` tells us we have a 3D array(bands × height × width), the Coordinate Reference System (CRS) anchors this data to a specific location on Earth, and the resolution information tells us the real-world size of each pixel. Finally, the per-band min/max values reveal the dynamic range of our spectral measurements. 

Although the pixel values in this image are stored as `uint8`, which supports a range from 0 to 255, all three bands in our dataset have maximum values of `254`. This usually happens because the largest possible value in the data type is often reserved as a `NoData` flag, marking pixels with no valid measurement. This upper limit is specific to this dataset, and the use of `NoData` flags is dataset dependent.

Image metadata is crucial because satellite imagery comes with precise spatial and spectral calibration. Each pixel value either represents actual physical measurements (like surface reflectance) at a specific geographic location, or a specific category that maps to a classification based on the dataset. Our model needs to understand not just "what values doe this pixel contain?" but also "where is this pixel?" and "what does this pixel represent?"

:::{.callout-note}
## Digital Numbers
The raw pixel values in many geospatial images are stored as integers to reduce file size and improve processing speed.  These integer values are called **Digital Numbers** (`DN`s).  

A `DN` is an instrument-recorded value that may need to be scaled or converted to physical units (such as reflectance or temperature) before analysis.
:::

### DN visualization


In [ ]:
# | echo: true
# | code-fold: true

# Display each band's raw DNs
num_bands = arr.shape[0]
fig, axes = plt.subplots(1, num_bands, figsize=(4*num_bands, 4))

# Handle case where we only have one band
if num_bands == 1:
    axes = [axes]

for i in range(num_bands):
    band = arr[i]
    data_min, data_max = float(band.min()), float(band.max())
    
    im = axes[i].imshow(band, cmap='viridis', vmin=data_min, vmax=data_max)
    axes[i].set_title(f"Band {i+1}: Raw Values\n(Range: {data_min:.0f} - {data_max:.0f})")
    axes[i].axis('off')
    
    # Add colorbar for each band
    plt.colorbar(im, ax=axes[i], label='Digital Number (DN)', shrink=0.8)

plt.tight_layout()
plt.show()

---

## 1.2  📐 Band Normalization

*Scaling bands to consistent ranges for stable model training*

Now that we've examined our raw data, we need to prepare it for neural network processing. While the raw digital numbers are meaningful for interpretation, they present several challenges when used directly as input to models. Different spectral bands often have vastly different value ranges, and **neural networks are sensitive to the scale of input features**.

### Why normalize geospatial data?

Normalization serves several critical purposes in geospatial machine learning:

**Scale Invariance**: Different spectral bands capture different physical phenomena and often have dramatically different value ranges. For example, near-infrared reflectance might range from 0-4000 DN, while visible bands range from 0-255 DN. Without normalization, the model would be dominated by the larger-valued bands.

**Training Stability**: Neural networks use gradient-based optimization, which works best when input features have similar scales. Large differences in feature magnitude can cause unstable gradients, slow convergence, or poor performance.

**Sensor Consistency**: When working with data from multiple sensors or time periods, normalization helps remove acquisition-specific biases while preserving the relative patterns that matter for learning.

**Activation Function Compatibility**: Many neural network activation functions (like `sigmoid` or `tanh`) work optimally when inputs are in specific ranges. Normalized inputs help ensure we're operating in the most effective part of these functions.

### Common normalization approaches

Let's examine the most common normalization strategies for geospatial data:

#### Min-Max Normalization (what we'll use)
Scales each band to a fixed range, typically [0,1]:
```
normalized = (value - min) / (max - min)
```

**Pros**: Preserves the distribution shape, guarantees output range  
**Cons**: Sensitive to outliers, range depends on specific dataset

#### Z-Score Standardization  
Centers data around zero with unit variance:
```
standardized = (value - mean) / std
```

**Pros**: Removes mean/variance effects, handles outliers better  
**Cons**: Output range is unbounded, can lose interpretability

#### Robust Scaling
Uses median and interquartile range instead of mean/std:
```
robust = (value - median) / (75th_percentile - 25th_percentile)
```

**Pros**: Very robust to outliers  
**Cons**: More complex, can compress dynamic range

For this exercise, we'll use min-max normalization because it's intuitive and preserves the relative structure of our data while ensuring all bands contribute equally to the model.


In [ ]:
# | echo: true
# Convert to float32 for numerical precision in calculations
arr = arr.astype(np.float32)
print(f"Data type after conversion: {arr.dtype}")

# Calculate per-band statistics
# Reshape to (bands, pixels) for easier computation
pixel_values = arr.reshape(arr.shape[0], -1)
mins = pixel_values.min(axis=1)
maxs = pixel_values.max(axis=1)
ranges = maxs - mins

print("Per-band statistics before normalization:")
for i in range(len(mins)):
    print(f"  Band {i+1}: min={mins[i]:.1f}, max={maxs[i]:.1f}, range={ranges[i]:.1f}")

In [ ]:
# | echo: true
# Apply min-max normalization: (x - min) / (max - min)
# Add small epsilon to prevent division by zero
eps = 1e-8
arr_norm = (arr - mins[:, None, None]) / (np.maximum(ranges, eps)[:, None, None])

print("\nNormalized to [0,1] range:")
print("Per-band min/max after normalization:")
for i, band in enumerate(arr_norm):
    print(f"  Band {i+1}: {float(band.min()):.3f} to {float(band.max()):.3f}")

### Visualizing the normalization effect

Let's examine how normalization changes the distribution of pixel values. This visualization helps us understand both what we've gained (comparable scales) and what we've preserved (relative patterns within each band).


In [ ]:
# | echo: true
# Compare distributions before and after normalization
fig, axes = plt.subplots(2, 3, figsize=(12, 6))

for i in range(min(3, arr.shape[0])):
    # Raw data histogram
    axes[0, i].hist(arr[i].ravel(), bins=50, color='gray', alpha=0.7, edgecolor='black')
    axes[0, i].set_title(f"Band {i+1}: Raw DNs")
    axes[0, i].set_xlabel("Digital Number")
    axes[0, i].set_ylabel("Frequency")
    
    # Normalized data histogram  
    axes[1, i].hist(arr_norm[i].ravel(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[1, i].set_title(f"Band {i+1}: Normalized [0,1]")
    axes[1, i].set_xlabel("Normalized Value")
    axes[1, i].set_ylabel("Frequency")
    axes[1, i].set_xlim(0, 1)  # Fix scale to show [0,1] range

plt.suptitle("Distribution Comparison: Raw vs. Normalized", fontsize=14)
plt.tight_layout()
plt.show()

Notice how the histograms show that while the scales have changed dramatically, the shape of each distribution—the relative patterns of light and dark areas—remains exactly the same. We've made the bands comparable without losing the information that distinguishes different materials and land cover types.

:::{.callout-tip}
## Historgram equalization

**What to notice**

- The raw data shows very different scales across bands (different x-axis ranges)
- After normalization, all bands span [0,1] but keep their unique distribution shapes
- This transformation makes bands equally "visible" to the neural network while preserving the spectral signatures that matter for classification
:::

Why this matters: Proper normalization is essential for stable training and ensures all spectral bands contribute meaningfully to the model's understanding of the landscape.

:::{.callout-tip}
## Deeper Dive: Normalization Method Comparison
For a comprehensive analysis of different normalization approaches used in state-of-the-art geospatial foundation models (including Prithvi, SatMAE, and Clay), see our detailed [Normalization Methods Comparison](../examples/normalization_comparison.qmd). This example compares computational performance, robustness to outliers, and practical trade-offs between five different normalization strategies.
:::

---

## 1.3 Tiling and patch extraction ✂️

*This section covers the patch extraction step in our roadmap: splitting large images into manageable "tokens" for model processing.*

### Why do we need patches?

When working with satellite imagery, we face a fundamental challenge: **scale mismatch**. A typical satellite image might be 10,000×10,000 pixels or larger, but neural networks work best with much smaller, consistent input sizes. But there's more to it than just computational limits. Patches serve several critical functions in geospatial machine learning:

**🧠 Cognitive Focus**: Just as humans focus on local areas when interpreting landscapes, neural networks learn more effectively when they can concentrate on coherent spatial neighborhoods rather than trying to process vast areas simultaneously.

**⚡ Computational Efficiency**: Smaller patches fit in GPU memory and enable parallel processing. We can train on batches of patches rather than one massive image at a time.

**🎯 Consistent Learning**: Fixed-size patches ensure the model sees consistent input dimensions, enabling it to learn spatial patterns at a specific scale.

**🔄 Data Augmentation**: From one large image, we can extract hundreds or thousands of training patches, dramatically increasing our dataset size.

### The tokenization analogy

The relationship between patches and tokenization runs deeper than you might first think. Let's explore this analogy:

| Aspect | Text Processing | Geospatial Processing |
|--------|----------------|----------------------|
| **Raw Input** | Documents, articles, books | Large satellite images, map tiles |
| **Atomic Unit** | Words/subwords → tokens | Pixels → patches |
| **Why Split?** | Models can't process infinite text | Models can't process arbitrarily large images |
| **Context Window** | Limited token context (e.g., 4K tokens) | Limited spatial context (e.g., 64×64 pixels) |
| **Semantic Coherence** | Tokens preserve word meaning | Patches preserve local spatial patterns |
| **Position Matters** | Word order affects meaning | Spatial arrangement affects interpretation |
| **Overlapping Context** | Sliding windows for long documents | Overlapping patches for spatial continuity |

### Visualizing the patch extraction process

Now we'll implement the patch extraction step from our roadmap. This transforms our normalized image into a collection of spatial "tokens" that our model can process.

### Understanding patch size trade-offs

Before we extract patches, it's important to understand how patch size affects what our model can learn:

| Patch Size | Pros | Cons | Best Use Cases |
|------------|------|------|----------------|
| **Small (16×16, 32×32)** | Fine detail, many samples, fast processing | Limited context, may miss large features | Urban analysis, crop monitoring |
| **Medium (64×64, 128×128)** | Good balance of detail and context | Moderate computational cost | General purpose, land cover mapping |
| **Large (256×256, 512×512)** | Rich spatial context, captures large features | Fewer samples, more memory intensive | Landscape analysis, climate modeling |

For our exercise, we'll use 64×64 patches—a sweet spot that captures meaningful spatial patterns while remaining computationally manageable.

### Step-by-step patch extraction

Now let's implement patch extraction with detailed visualization of each step:

In [ ]:
#| echo: true
# First, let's examine our normalized image dimensions
C, H, W = arr_norm.shape
print(f"Input image shape: {C} bands × {H} height × {W} width pixels")
print(f"Using patch size: {PATCH_SIZE}×{PATCH_SIZE} pixels")

# Calculate how many complete patches we can extract
ph = H // PATCH_SIZE  # patches vertically
pw = W // PATCH_SIZE  # patches horizontally
print(f"Complete patches: {ph} rows × {pw} columns = {ph * pw} total patches")

# Calculate the cropped dimensions (we may lose edge pixels)
Hc, Wc = ph * PATCH_SIZE, pw * PATCH_SIZE
print(f"Cropped image size: {Hc}×{Wc} (original: {H}×{W})")
print(f"Edge pixels lost: {H-Hc} height, {W-Wc} width")

In [ ]:
#| echo: true
# Step 1: Crop the image to fit complete patches
arr_c = arr_norm[:, :Hc, :Wc]
print(f"Cropped array shape: {arr_c.shape}")

# Step 2: Visualize the grid overlay on the original image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Show original image with patch grid overlay
if C >= 3:
    rgb = np.transpose(arr_norm[:3], (1, 2, 0))
else:
    rgb = np.stack([arr_norm[0]]*3, axis=-1)

ax1.imshow(np.clip(rgb, 0, 1))
ax1.set_title("Original Image with Patch Grid")

# Draw grid lines to show patch boundaries
for i in range(ph + 1):
    ax1.axhline(y=i * PATCH_SIZE - 0.5, color='red', linewidth=1, alpha=0.7)
for j in range(pw + 1):
    ax1.axvline(x=j * PATCH_SIZE - 0.5, color='red', linewidth=1, alpha=0.7)

# Show cropped version
if C >= 3:
    rgb_cropped = np.transpose(arr_c[:3], (1, 2, 0))
else:
    rgb_cropped = np.stack([arr_c[0]]*3, axis=-1)

ax2.imshow(np.clip(rgb_cropped, 0, 1))
ax2.set_title("Cropped Image (Complete Patches Only)")

plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
# Step 3: The magical reshape operation that extracts patches
# This is the core of patch extraction - let's break it down step by step

print("🔄 Patch extraction transformation:")
print(f"1. Input shape: {arr_c.shape} → (bands, height, width)")

# Reshape to separate patch rows and columns
reshaped = arr_c.reshape(C, ph, PATCH_SIZE, pw, PATCH_SIZE)
print(f"2. After reshape: {reshaped.shape} → (bands, patch_rows, patch_height, patch_cols, patch_width)")

# Transpose to group patches together
transposed = reshaped.transpose(1, 3, 0, 2, 4)
print(f"3. After transpose: {transposed.shape} → (patch_rows, patch_cols, bands, patch_height, patch_width)")

# Final reshape to get individual patches
patches = transposed.reshape(ph * pw, C, PATCH_SIZE, PATCH_SIZE)
print(f"4. Final patches: {patches.shape} → (num_patches, bands, patch_height, patch_width)")

print(f"\n✅ Successfully extracted {patches.shape[0]} patches!")

In [ ]:
#| echo: true
# Step 4: Visualize sample patches to see what we've extracted
nshow = min(6, len(patches))
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for i in range(nshow):
    p = patches[i]
    
    # Create RGB visualization (handle cases with fewer than 3 bands)
    if C >= 3:
        rgb = np.transpose(p[:3], (1, 2, 0))
    else:
        rgb = np.stack([p[0]]*3, axis=-1)
    
    axes[i].imshow(np.clip(rgb, 0, 1))
    
    # Calculate patch position in the grid
    row = i // pw
    col = i % pw
    axes[i].set_title(f"Patch {i}\nGrid position: ({row}, {col})")
    axes[i].axis('off')

plt.suptitle("Sample Patches: Our Image 'Tokens'", fontsize=14)
plt.tight_layout()
plt.show()

### Understanding what we've created

Each patch is now a self-contained "token" that represents a local region of our satellite image. Just like how words in a sentence carry meaning individually but also depend on context, these patches contain local spatial patterns while maintaining relationships to their neighboring patches.

:::{.callout-note}
## Key Transformation
We've transformed one large image `(3, H, W)` into `{ph × pw}` smaller images, each of shape `(3, 64, 64)`. This is exactly analogous to how tokenization transforms one long text into many smaller tokens that a model can process efficiently.
:::

Why this matters: Patches become our fundamental unit of analysis—each patch can be processed independently while preserving spatial relationships for the model to learn landscape patterns.

---


## 1.4 Assigning patch IDs & metadata 🏷️

*This section covers the metadata step in our roadmap: giving each patch an identity and spatial context.*

"Every patch needs an identity — not just for bookkeeping, but to remember where and when it came from."

- Generate unique IDs for patches.
- Store spatial metadata (bounding box, CRS) and acquisition info.
- Print one sample metadata record.


In [ ]:
#| echo: true
from rasterio.transform import xy

# Compute patch bounding boxes (min lon/lat, max lon/lat approx)
ids = []
meta = []
idx = 0
for i in range(ph):
    for j in range(pw):
        pid = f"patch_{i:03d}_{j:03d}"
        # top-left pixel (row=i*P, col=j*P)
        r0, c0 = i*PATCH_SIZE, j*PATCH_SIZE
        r1, c1 = r0 + PATCH_SIZE - 1, j * PATCH_SIZE + PATCH_SIZE - 1
        (x0, y0) = xy(transform, r0, c0)
        (x1, y1) = xy(transform, r1, c1)
        bbox = (min(x0,x1), min(y0,y1), max(x0,x1), max(y0,y1))
        ids.append(pid)
        meta.append({"id": pid, "bbox": bbox, "crs": str(crs)})
        idx += 1

print("total ids:", len(ids))
print("sample record:", meta[0])

Why this matters: metadata lets us track location/time and avoid leakage when splitting data.

---

## 1.5 Adding special context tokens (location, time, sensor) 🌍

*This section covers the spatial-temporal context step in our roadmap: encoding where and when each patch was captured.*

"In addition to pixel values, our model benefits from knowing where and when the data was collected, and by which sensor."

- Encode lat/lon as continuous features or sinusoidal embeddings.
- Encode acquisition date as day‑of‑year + year.
- Show how these features attach to patches.


In [ ]:
#| echo: true
import datetime as dt

# Example acquisition date (placeholder metadata)
acq_date = dt.date(2023, 7, 15)
doy = acq_date.timetuple().tm_yday
year = acq_date.year

# Use patch centers for location features
centers = []
for i in range(ph):
    for j in range(pw):
        r, c = i*PATCH_SIZE + PATCH_SIZE//2, j*PATCH_SIZE + PATCH_SIZE//2
        x_c, y_c = xy(transform, r, c)
        centers.append((x_c, y_c))
centers = np.array(centers, dtype=np.float32)

# Simple scaled features (you can swap for sinusoidal if desired)
loc_feat = (centers - centers.mean(0)) / (centers.std(0) + 1e-6)
time_feat = np.array([doy/366.0, (year-2000)/50.0], dtype=np.float32)  # scale to ~[0,1]
print("loc_feat shape:", loc_feat.shape, "time_feat:", time_feat.tolist())

Why this matters: models need context to learn spatial/temporal patterns.

---

## 1.6 Dimensionality reduction for spectral bands (analogy to BPE)

“Sometimes we have more bands than the model needs. We can reduce the input size while keeping the most useful information.”

- Demonstrate PCA or band selection.
- Compare original vs. reduced band profiles.


In [ ]:
#| echo: true
# PCA via SVD on (pixels, bands) for speed; sample a subset of pixels
C, Hc, Wc = arr_c.shape
pixels = arr_c.reshape(C, -1).T  # (N, C)
N = pixels.shape[0]
sub = min(5000, N)
idx = np.random.default_rng(RNG_SEED).choice(N, sub, replace=False)
Xsub = pixels[idx]

# Center
mu = Xsub.mean(0, keepdims=True)
Xc = Xsub - mu
U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
explained = (S**2) / (S**2).sum()
keep = min(3, C)
Wred = Vt[:keep].T  # (C, keep)

print("bands:", C, "keep:", keep, "explained_var@keep:", float(explained[:keep].sum()))

# Project full image to reduced components per pixel
red_pixels = (pixels - mu).dot(Wred)  # (N, keep)
red_stack = red_pixels.T.reshape(keep, Hc, Wc)
print("reduced stack shape:", red_stack.shape)

Why this matters: reducing input size speeds training and may denoise bands.

---

## 1.7 Sampling training patches (sliding windows, spatial splits)

“Rather than using the whole dataset at once, we’ll sample patches in a way that keeps training efficient and avoids data leakage.”

- Show random vs. sliding‑window sampling.
- Highlight spatial/temporal separation for validation/test.


In [ ]:
#| echo: true
num_patches = patches.shape[0]
all_idx = np.arange(num_patches)
# Simple spatial split by rows: top 80% train, bottom 20% val
rows = np.repeat(np.arange(ph), pw)
train_mask = rows < int(0.8*ph)
train_idx = all_idx[train_mask]
val_idx = all_idx[~train_mask]

rng = np.random.default_rng(RNG_SEED)
rng.shuffle(train_idx)
print("num_patches:", num_patches, "train:", len(train_idx), "val:", len(val_idx))
print("example train idx:", train_idx[:8].tolist())

Why this matters: proper splits reduce spatial leakage and inflated metrics.

---

## 1.8 Creating patch embeddings 🧠

*This section covers the patch embeddings step in our roadmap: transforming spatial patches into the vector space where models operate.*

"This is where pixels meet the model: we project each patch into an embedding space where the model can learn patterns."

- Flatten + linear layer (minimal version).
- Print embedding tensor shapes.
- Show a few example embedding vectors.


In [ ]:
#| echo: true
try:
    import torch
    torch.manual_seed(RNG_SEED)
    B, C, P, _ = patches.shape
    input_dim = C * P * P
    embed_dim = 64
    proj = torch.nn.Linear(input_dim, embed_dim)

    # take a tiny batch of patches
    sample = torch.from_numpy(patches[:4]).reshape(4, -1).float()
    emb = proj(sample)
    print("input_dim:", input_dim, "embed_dim:", embed_dim, "emb shape:", tuple(emb.shape))
    print("first row (rounded):", [float(x) for x in emb[0][:8].detach().numpy().round(3)])
except Exception as e:
    print("Torch not available; skipping embeddings", e)

Why this matters: embeddings are the model’s operating space for learning patterns.

---

## 1.9 Encoding spatial & temporal positions 📍

*This section covers the final step in our roadmap: adding positional encodings so models understand spatial and temporal relationships.*

"Models don't know where a patch sits in space or time unless we tell them — positional encodings add this crucial context."

- Create 2D sinusoidal spatial encodings (row/col).
- Add temporal encodings (day‑of‑year).
- Combine with patch embeddings and inspect results.


In [ ]:
#| echo: true
# 2D sinusoidal positional encodings for (row, col)
rows_grid = np.arange(ph)[:,None].repeat(pw,1)
cols_grid = np.arange(pw)[None,:].repeat(ph,0)
pos2d = np.stack([rows_grid.ravel(), cols_grid.ravel()], axis=1).astype(np.float32)

# Scale rows/cols to [0,1]
pos2d = (pos2d - pos2d.min(0)) / (pos2d.ptp(0) + 1e-6)

# Simple sin/cos features
pos_feat = np.concatenate([np.sin(2*np.pi*pos2d), np.cos(2*np.pi*pos2d)], axis=1)
# time features from earlier (same for all patches here)
time_feat_full = np.repeat(time_feat[None,:], len(pos_feat), axis=0)

print("pos_feat shape:", pos_feat.shape, "time_feat_full shape:", time_feat_full.shape)

In [ ]:
#| echo: true
# Combine: embedding + positional + time features (toy example)
try:
    comb = np.concatenate([
        emb.detach().numpy(),         # (4, 64) toy batch
        pos_feat[:4],                 # (4, 4)
        time_feat_full[:4]            # (4, 2)
    ], axis=1)
    print("combined features shape:", comb.shape)
except Exception as e:
    print("Combine skipped:", e)

Why this matters: spatial/temporal encodings provide essential context alongside pixel content.

---

## Conclusion
- We inspected raster metadata (CRS, resolution, bands) and normalized bands
- We extracted fixed‑size patches, attached IDs/metadata, and added location/time features
- We created minimal patch embeddings and combined them with positional/temporal encodings

## Resources
- Quarto: https://quarto.org/docs
- Rasterio: https://rasterio.readthedocs.io/
- NumPy: https://numpy.org/doc/
- PyTorch: https://pytorch.org/docs/stable/index.html
- Vision Transformers (ViT): https://arxiv.org/abs/2010.11929